# Load & Analyze tld_reseller model performance comparison

In [ ]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)

In [47]:
install.packages("pkgcond")

Installing package into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)



In [48]:
library(dplyr)
library(data.table)
library(partykit)
library(tictoc)
library(caret)
library(e1071)
library(randomForest)
library(ranger)

#for 3d plotting
library(akima)
library(plotly)

# for prep data
library(rPython)
library(stringr)
library(pbapply)
library(stringdist)
library(data.table)
library(dominanceanalysis)

library(pkgcond)

getwd()


Attaching package: ‘pkgcond’


The following object is masked from ‘package:dplyr’:

    collapse




[1] "/home/jupyter/local/Domains_202003/scripts/phaseII_05_moredata"

In [1]:

source('../orig/functions.R')

source('../orig/functions_models.R')

source('../phaseII_03_forest/functions_eval.R')


source('../phaseII_03_forest/load_prep_data_expiry.R')

# expiry_train_prepped_2_1 (list, less df's w/ 0 obs)
# expiry_test_prepped_2_1  (list, less df's w/ 0 obs)
# expiry_train_df_1 (above, rbound)
# expiry_test_df_1 (above, rbound)
# expiry_train_df_sub (subset vars)                                        
# expiry_test_df_sub (subset vars)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




## List tld_reseller labels

In [4]:
tld_reseller_lookup_df = expiry_train_df_1 %>% group_by(tld_registrar_index, reseller) %>% tally() %>% arrange(desc(n))
tld_reseller_lookup_df %>% head()

tld_registrar_index,reseller,n
<chr>,<chr>,<int>
sitegmo,gmo,106569
funalibaba,alibaba,72690
sitenamecheap,namecheap,53536
pwnamecheap,namecheap,50470
onlinego daddy,go daddy,42162
sitegandi sas,gandi sas,35309


## Load script results

In [5]:
ls()

[1] "apply_standard_adjustments"         "basic_prep_domain_data"            
 [3] "build_model_first_renewal"          "build_model_first_renewal_agg"     
 [5] "build_model_first_renewal_agg_plus" "build_model_first_renewal_reg"     
 [7] "build_model_second_renewal"         "calc_auc"                          
 [9] "change.colnames"                    "chart_lift"                        
[11] "expiry_test_df_1"                   "expiry_test_df_sub"                
[13] "expiry_test_prepped_2"              "expiry_test_prepped_2_1"           
[15] "expiry_train_df_1"                  "expiry_train_df_sub"               
[17] "expiry_train_prepped_2"             "expiry_train_prepped_2_1"          
[19] "get_cluster_data"                   "get_cluster_large"                 
[21] "get_df_npv"                         "get_domain_npv"                    
[23] "get_domain_type"                    "get_pattern_score"                 
[25] "get_renewal_status"                 "intersect2"                        
[27] "list_predict_first_renewal"         "list_predict_second_renewal"       
[29] "mass_build_model_first_renewal"     "mass_build_model_second_renewal"   
[31] "mass_predict_first_renewal"         "mass_predict_second_renewal"       
[33] "mass_prep_data"                     "na.zero"                           
[35] "newreg_data_prep"                   "newreg_data_rename"                
[37] "newreg_data_select"                 "plot_gains"                        
[39] "plot_multigains"                    "plot_multigains_file"              
[41] "plot_multigains_res"                "predict_first_renewal"             
[43] "predict_first_renewal_agg"          "predict_first_renewal_agg_plus"    
[45] "predict_first_renewal_reg"          "predict_second_renewal"            
[47] "prep_domain_data"                   "save_object"                       
[49] "tld_reseller_list"                  "tld_reseller_lookup_df"            
[51] "unzip.file"

In [11]:
load("../../data/tld_reseller_compare_list_0001_0817")
tld_reseller_compare_list_0001_0817 <- return_list
load("../../data/tld_reseller_compare_list_0001_0817_seg_rf")
tld_reseller_compare_list_0001_0817_seg_rf <- return_list
load("../../data/tld_reseller_compare_list_0818_1723.RData")
tld_reseller_compare_list_0818_1723.RData <- return_list

In [16]:
length(tld_reseller_compare_list_0001_0817)
length(tld_reseller_compare_list_0001_0817[[1]])
names(tld_reseller_compare_list_0001_0817[[1]])

[1] 817

[1] 6

[1] "pred_df_seg2_glm" "pred_df_seg_glm"  "pred_df_agg_glm"  "pred_df_seg2_rf" 
[5] "pred_df_seg_rf"   "pred_df_agg_rf"

In [18]:
length(tld_reseller_compare_list_0001_0817_seg_rf)
length(tld_reseller_compare_list_0001_0817_seg_rf[[1]])
names(tld_reseller_compare_list_0001_0817_seg_rf[[1]])

[1] 817

[1] 1

[1] "pred_df_seg_rf"

In [14]:
length(tld_reseller_compare_list_0818_1723.RData)

[1] 906

## Munge List of Lists to List of Dataframes

In [141]:
test_funct <- function(list_elem){
    df <- bind_cols(list_elem)
    if (ncol(df)>6) {
        df <- df[c(1,2,4,6,8,10,12)]
        names(df) <- c("actual",names(list_elem))
    } else {
        df <- df
        df$actual <- NA
        df <- df %>% select(actual, everything())
    }
        

    return(df)
}

In [142]:
suppress_messages(compare_df_list_0001_0817 <- lapply(tld_reseller_compare_list_0001_0817, test_funct))

In [143]:
for (i in 1:length(tld_reseller_compare_list_0001_0817)){
    compare_df_list_0001_0817[[i]] <- cbind(compare_df_list_0001_0817[[i]], tld_reseller_compare_list_0001_0817_seg_rf[[i]])
    compare_df_list_0001_0817[[i]] <- compare_df_list_0001_0817[[i]][c(1:5,7,9)]
    names(compare_df_list_0001_0817[[i]]) <- c('actual','pred_df_seg2_glm','pred_df_seg_glm','pred_df_agg_glm','pred_df_seg2_rf',
                                          'pred_df_agg_rf','pred_df_seg_rf')
    compare_df_list_0001_0817[[i]] <- compare_df_list_0001_0817[[i]][c('actual',
                                                             'pred_df_seg2_glm','pred_df_seg_glm','pred_df_agg_glm',
                                                             'pred_df_seg2_rf','pred_df_seg_rf','pred_df_agg_rf')]
}

In [144]:
suppress_messages(compare_df_list_0818_1723 <- lapply(tld_reseller_compare_list_0818_1723.RData, test_funct))

In [149]:
length(compare_df_list_0001_0817)
length(compare_df_list_0818_1723)

compare_df_list_0001_1723 <- append(compare_df_list_0001_0817,compare_df_list_0818_1723)
length(compare_df_list_0001_1723)


[1] 817

[1] 906

[1] 1723